# Notebook 12: RDLS HEVL Integration

**Purpose**: Integrate HEVL extractions (from Notebooks 09-11) with general metadata records (from Notebook 06) to produce complete RDLS v0.3 records.

**Approach**:
1. Load NB 06 general metadata JSONs as the base (authoritative general metadata)
2. Load HEVL extraction CSVs (from NB 09-11) to determine component flags per dataset
3. Load HEVL extraction JSONs (from NB 09-11) with detailed HEVL blocks
4. Merge HEVL blocks into base records, update `risk_data_type`
5. Generate final integrated RDLS JSON files

**Key principle**: NB 12 does NOT rebuild general metadata or HEVL blocks. It merges
pre-built outputs from upstream notebooks (NB 06 for general, NB 09-11 for HEVL).

**Author**: Benny Istanto/Risk Data Librarian/GFDRR  
**Version**: 2026.2

---

## 1. Setup

In [1]:
"""
1.1 Import Dependencies
"""

import json
from pathlib import Path
from datetime import datetime
from typing import Dict, List, Optional, Any, Tuple
from copy import deepcopy

import pandas as pd
import numpy as np

try:
    from tqdm.notebook import tqdm
    HAS_TQDM = True
except ImportError:
    HAS_TQDM = False

pd.set_option('display.max_columns', None)

print(f"Notebook started: {datetime.now().isoformat()}")

Notebook started: 2026-02-11T18:08:41.937336


In [2]:
"""
1.2 Configure Paths and Output Settings
"""

NOTEBOOK_DIR = Path.cwd()
BASE_DIR = NOTEBOOK_DIR.parent if NOTEBOOK_DIR.name == 'notebook' else NOTEBOOK_DIR

# ── Output cleanup mode ───────────────────────────────────────────────
# Controls what happens to old output files when this notebook is re-run.
#   "replace" - Auto-delete old outputs and continue (default)
#   "prompt"  - Show what will be deleted, ask user to confirm
#   "skip"    - Keep old files, write new on top (may leave orphans)
#   "abort"   - Stop if old outputs exist (for CI/automated runs)
CLEANUP_MODE = "replace"

# NB 06 output: general metadata records (the BASE for integration)
RECORDS_DIR = BASE_DIR / 'hdx_dataset_metadata_dump' / 'rdls' / 'records'

# NB 06 output: index JSONL mapping HDX UUIDs to NB 06 filenames
INDEX_JSONL = BASE_DIR / 'hdx_dataset_metadata_dump' / 'rdls' / 'index' / 'rdls_index.jsonl'

# NB 09-11 output: HEVL extraction CSVs and JSONs
EXTRACTED_DIR = BASE_DIR / 'hdx_dataset_metadata_dump' / 'rdls' / 'extracted'

# Extraction CSVs (separate files from revised NB 09-11)
HAZARD_CSV = EXTRACTED_DIR / 'hazard_extraction_results.csv'
EXPOSURE_CSV = EXTRACTED_DIR / 'exposure_extraction_results.csv'
VULNERABILITY_CSV = EXTRACTED_DIR / 'vulnerability_extraction_results.csv'
LOSS_CSV = EXTRACTED_DIR / 'loss_extraction_results.csv'

# Output paths
OUTPUT_DIR = BASE_DIR / 'hdx_dataset_metadata_dump' / 'rdls' / 'integrated'
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

print(f"Base: {BASE_DIR}")
print(f"NB 06 records: {RECORDS_DIR}")
print(f"NB 06 index:   {INDEX_JSONL}  (exists={INDEX_JSONL.exists()})")
print(f"Extracted:     {EXTRACTED_DIR}")
print(f"Output:        {OUTPUT_DIR}")
print(f"Cleanup mode:  {CLEANUP_MODE}")

Base: /mnt/c/Users/benny/OneDrive/Documents/Github/hdx-metadata-crawler
NB 06 records: /mnt/c/Users/benny/OneDrive/Documents/Github/hdx-metadata-crawler/hdx_dataset_metadata_dump/rdls/records
NB 06 index:   /mnt/c/Users/benny/OneDrive/Documents/Github/hdx-metadata-crawler/hdx_dataset_metadata_dump/rdls/index/rdls_index.jsonl  (exists=True)
Extracted:     /mnt/c/Users/benny/OneDrive/Documents/Github/hdx-metadata-crawler/hdx_dataset_metadata_dump/rdls/extracted
Output:        /mnt/c/Users/benny/OneDrive/Documents/Github/hdx-metadata-crawler/hdx_dataset_metadata_dump/rdls/integrated
Cleanup mode:  replace


In [3]:
"""
1.3 Load NB 06 Index and General Metadata Records

The NB 06 index JSONL is the BRIDGE between:
  - HDX dataset UUIDs (used in HEVL extraction CSVs)
  - NB 06 record filenames (slug-based, e.g., rdls_exp-hdx_3is_eth_...)

We load the index first to build a UUID→filename mapping,
then load the actual NB 06 JSON records.
"""

def load_nb06_index(index_path: Path) -> pd.DataFrame:
    """
    Load rdls_index.jsonl produced by NB 06.
    
    Each line contains: dataset_id (HDX UUID), rdls_id, filename, etc.
    """
    if not index_path.exists():
        print(f"  ERROR: Index not found at {index_path}")
        return pd.DataFrame()
    
    records = []
    with open(index_path, 'r', encoding='utf-8') as f:
        for line in f:
            line = line.strip()
            if line:
                try:
                    records.append(json.loads(line))
                except json.JSONDecodeError:
                    pass
    
    df = pd.DataFrame(records)
    print(f"Loaded NB 06 index: {len(df):,} entries")
    print(f"  Columns: {list(df.columns)}")
    return df


def load_nb06_records(records_dir: Path, index_df: pd.DataFrame) -> Dict[str, Dict]:
    """
    Load NB 06 RDLS JSON records, indexed by full HDX dataset UUID.
    
    Uses the index JSONL to map UUID -> filename, then loads each JSON.
    """
    records = {}
    missing = 0
    
    for _, row in index_df.iterrows():
        hdx_uuid = row.get('dataset_id', '')
        filename = row.get('filename', '')
        
        if not hdx_uuid or not filename:
            continue
        
        filepath = records_dir / filename
        if not filepath.exists():
            missing += 1
            continue
        
        try:
            with open(filepath, 'r', encoding='utf-8') as f:
                data = json.load(f)
            
            ds = data.get('datasets', [{}])[0]
            
            records[hdx_uuid] = {
                'filepath': filepath,
                'data': data,
                'rdls_id': ds.get('id', ''),
                'title': ds.get('title', ''),
            }
        except Exception as e:
            print(f"  Error loading {filename}: {e}")
    
    print(f"Loaded {len(records):,} NB 06 records indexed by HDX UUID")
    if missing:
        print(f"  ({missing} index entries had no matching JSON file)")
    return records


# Load index, then records
df_nb06_index = load_nb06_index(INDEX_JSONL)
nb06_records = load_nb06_records(RECORDS_DIR, df_nb06_index)

Loaded NB 06 index: 13,152 entries
  Columns: ['dataset_id', 'rdls_id', 'filename', 'risk_data_type', 'spatial_scale', 'countries_count', 'license_raw', 'orgtoken', 'hazard_suffix', 'organization_token', 'iso3', 'hazard_types', 'blocked', 'blocked_reasons']
Loaded 13,152 NB 06 records indexed by HDX UUID


## 2. Load HEVL Extraction Results

In [4]:
"""
2.1 Load HEVL Extraction CSVs

These CSVs tell us WHICH datasets have WHICH components.
- hazard_extraction_results.csv  (NB 09): has_hazard flag + hazard_types
- exposure_extraction_results.csv (NB 10): has_exposure flag + categories
- vulnerability_extraction_results.csv (NB 11): has_vulnerability flag
- loss_extraction_results.csv (NB 11): has_loss flag
"""

def load_csv_safe(path: Path, label: str) -> Optional[pd.DataFrame]:
    """Load CSV if it exists, return None otherwise."""
    if path.exists():
        df = pd.read_csv(path, low_memory=False)
        print(f"  {label}: {len(df):,} records, columns={list(df.columns)}")
        return df
    print(f"  {label}: NOT FOUND at {path.name}")
    return None

print("Loading HEVL extraction CSVs...")
df_hazard = load_csv_safe(HAZARD_CSV, "Hazard (NB 09)")
df_exposure = load_csv_safe(EXPOSURE_CSV, "Exposure (NB 10)")
df_vuln = load_csv_safe(VULNERABILITY_CSV, "Vulnerability (NB 11)")
df_loss = load_csv_safe(LOSS_CSV, "Loss (NB 11)")

Loading HEVL extraction CSVs...
  Hazard (NB 09): 26,246 records, columns=['id', 'title', 'organization', 'hazard_types', 'process_types', 'analysis_type', 'return_periods', 'intensity_measures', 'calculation_method', 'overall_confidence', 'has_hazard']
  Exposure (NB 10): 26,246 records, columns=['id', 'title', 'organization', 'categories', 'category_count', 'taxonomy', 'overall_confidence', 'has_exposure']
  Vulnerability (NB 11): 26,246 records, columns=['id', 'title', 'organization', 'has_functions', 'function_types', 'has_socio_economic', 'socio_indicators', 'overall_confidence', 'has_vulnerability']
  Loss (NB 11): 26,246 records, columns=['id', 'title', 'organization', 'has_loss', 'loss_count', 'loss_signal_types', 'hazard_types', 'asset_categories', 'overall_confidence']


In [5]:
"""
2.2 Build Merged HEVL Flags DataFrame

Merge all four CSVs on 'id' to get a single row per dataset
with boolean has_hazard / has_exposure / has_vulnerability / has_loss flags.
"""

def merge_hevl_flags(
    haz_df: Optional[pd.DataFrame],
    exp_df: Optional[pd.DataFrame],
    vul_df: Optional[pd.DataFrame],
    los_df: Optional[pd.DataFrame],
) -> pd.DataFrame:
    """Merge HEVL detection flags into one DataFrame keyed by HDX dataset id."""
    
    frames = []
    
    if haz_df is not None and 'id' in haz_df.columns:
        h = haz_df[['id', 'has_hazard']].copy()
        h['has_hazard'] = h['has_hazard'].astype(bool)
        frames.append(h)
    
    if exp_df is not None and 'id' in exp_df.columns:
        e = exp_df[['id', 'has_exposure']].copy()
        e['has_exposure'] = e['has_exposure'].astype(bool)
        frames.append(e)
    
    if vul_df is not None and 'id' in vul_df.columns:
        v = vul_df[['id', 'has_vulnerability']].copy()
        v['has_vulnerability'] = v['has_vulnerability'].astype(bool)
        frames.append(v)
    
    if los_df is not None and 'id' in los_df.columns:
        l = los_df[['id', 'has_loss']].copy()
        l['has_loss'] = l['has_loss'].astype(bool)
        frames.append(l)
    
    if not frames:
        return pd.DataFrame(columns=['id', 'has_hazard', 'has_exposure', 'has_vulnerability', 'has_loss'])
    
    merged = frames[0]
    for f in frames[1:]:
        merged = merged.merge(f, on='id', how='outer')
    
    # Fill missing flags with False
    for col in ['has_hazard', 'has_exposure', 'has_vulnerability', 'has_loss']:
        if col not in merged.columns:
            merged[col] = False
        merged[col] = merged[col].fillna(False).astype(bool)
    
    return merged

df_flags = merge_hevl_flags(df_hazard, df_exposure, df_vuln, df_loss)
print(f"\nMerged flags: {len(df_flags):,} unique datasets")

# Filter to datasets with at least one HEVL signal
any_hevl = df_flags['has_hazard'] | df_flags['has_exposure'] | df_flags['has_vulnerability'] | df_flags['has_loss']
df_hevl = df_flags[any_hevl].copy()
print(f"With any HEVL signal: {len(df_hevl):,}")


Merged flags: 26,246 unique datasets
With any HEVL signal: 22,473


In [6]:
"""
2.3 Analyze HEVL Coverage
"""

print("=" * 60)
print("HEVL EXTRACTION COVERAGE")
print("=" * 60)

total = len(df_flags)
h_count = df_flags['has_hazard'].sum()
e_count = df_flags['has_exposure'].sum()
v_count = df_flags['has_vulnerability'].sum()
l_count = df_flags['has_loss'].sum()

print(f"\nTotal datasets scanned: {total:,}")
print(f"\nComponent Detection:")
print(f"  Hazard (H):        {h_count:>6,} ({h_count/total*100:>5.1f}%)")
print(f"  Exposure (E):      {e_count:>6,} ({e_count/total*100:>5.1f}%)")
print(f"  Vulnerability (V): {v_count:>6,} ({v_count/total*100:>5.1f}%)")
print(f"  Loss (L):          {l_count:>6,} ({l_count/total*100:>5.1f}%)")

# HEVL combination analysis
df_flags['hevl_combo'] = (
    df_flags['has_hazard'].apply(lambda x: 'H' if x else '-') +
    df_flags['has_exposure'].apply(lambda x: 'E' if x else '-') +
    df_flags['has_vulnerability'].apply(lambda x: 'V' if x else '-') +
    df_flags['has_loss'].apply(lambda x: 'L' if x else '-')
)

print(f"\nHEVL Combinations (top 15):")
for combo, count in df_flags['hevl_combo'].value_counts().head(15).items():
    pct = count / total * 100
    print(f"  {combo}: {count:>6,} ({pct:>5.1f}%)")

HEVL EXTRACTION COVERAGE

Total datasets scanned: 26,246

Component Detection:
  Hazard (H):         3,208 ( 12.2%)
  Exposure (E):      21,631 ( 82.4%)
  Vulnerability (V):  5,327 ( 20.3%)
  Loss (L):           5,771 ( 22.0%)

HEVL Combinations (top 15):
  -E--: 11,091 ( 42.3%)
  ----:  3,773 ( 14.4%)
  -EV-:  3,164 ( 12.1%)
  -E-L:  2,709 ( 10.3%)
  -EVL:  1,952 (  7.4%)
  HE--:  1,800 (  6.9%)
  HE-L:    764 (  2.9%)
  H---:    473 (  1.8%)
  ---L:    289 (  1.1%)
  HEV-:    114 (  0.4%)
  --V-:     60 (  0.2%)
  HEVL:     37 (  0.1%)
  H--L:     20 (  0.1%)


## 3. Load HEVL Extraction JSONs

In [7]:
"""
3.1 Load HEVL JSON Files by Component Type

NB 09-11 each produce sample RDLS JSONs with detailed HEVL blocks:
  - rdls_hzd-hdx_*.json  (hazard blocks from NB 09)
  - rdls_exp-hdx_*.json  (exposure blocks from NB 10)
  - rdls_vln-hdx_*.json  (vulnerability blocks from NB 11)
  - rdls_lss-hdx_*.json  (loss blocks from NB 11)

Each JSON has structure: {"datasets": [{"hazard": {...}, ...}]}
We extract the HEVL-specific keys and index by FULL HDX UUID.

NOTE: HEVL JSON filenames use 8-char UUID prefixes (e.g., rdls_hzd-hdx_0013fbe3.json).
We use the NB 06 index to resolve these to full 36-char UUIDs.
"""

import re

def build_uuid8_to_full(index_df: pd.DataFrame) -> Dict[str, str]:
    """Build mapping from 8-char UUID prefix to full 36-char UUID."""
    mapping = {}
    if 'dataset_id' in index_df.columns:
        for uuid_full in index_df['dataset_id'].dropna():
            uuid8 = uuid_full[:8]
            mapping[uuid8] = uuid_full
    print(f"UUID8→full mapping: {len(mapping):,} entries")
    return mapping


def load_hevl_jsons(
    extracted_dir: Path,
    uuid8_to_full: Dict[str, str],
) -> Dict[str, Dict[str, Any]]:
    """
    Load all HEVL JSONs from extracted_dir, indexed by FULL HDX UUID.
    
    Returns dict: full_hdx_uuid -> {
        'hazard': {...} or absent,
        'exposure': [...] or absent,
        'vulnerability': {...} or absent,
        'loss': {...} or absent,
    }
    """
    hevl_blocks = {}  # full_uuid -> component blocks
    unresolved = 0
    
    # Component key per file prefix
    prefix_to_key = {
        'rdls_hzd': 'hazard',
        'rdls_exp': 'exposure',
        'rdls_vln': 'vulnerability',
        'rdls_lss': 'loss',
    }
    
    for filepath in sorted(extracted_dir.glob('rdls_*.json')):
        try:
            # Determine component type from filename prefix
            stem = filepath.stem
            component_key = None
            for prefix, key in prefix_to_key.items():
                if stem.startswith(prefix):
                    component_key = key
                    break
            
            if component_key is None:
                continue
            
            # Extract 8-char UUID from filename: rdls_hzd-hdx_0013fbe3 -> 0013fbe3
            match = re.search(r'hdx_([a-f0-9]{8})$', stem)
            if not match:
                continue
            
            uuid8 = match.group(1)
            full_uuid = uuid8_to_full.get(uuid8)
            
            if not full_uuid:
                unresolved += 1
                continue
            
            with open(filepath, 'r', encoding='utf-8') as f:
                data = json.load(f)
            
            ds = data.get('datasets', [{}])[0]
            
            if full_uuid not in hevl_blocks:
                hevl_blocks[full_uuid] = {}
            
            # Extract the component-specific block
            block = ds.get(component_key)
            if block is not None:
                hevl_blocks[full_uuid][component_key] = block
                
        except Exception as e:
            print(f"  Error loading {filepath.name}: {e}")
    
    # Count by component
    counts = {'hazard': 0, 'exposure': 0, 'vulnerability': 0, 'loss': 0}
    for uuid_full, blocks in hevl_blocks.items():
        for key in counts:
            if key in blocks:
                counts[key] += 1
    
    print(f"Loaded HEVL JSON blocks for {len(hevl_blocks)} datasets:")
    for key, count in counts.items():
        print(f"  {key}: {count} blocks")
    if unresolved:
        print(f"  ({unresolved} HEVL JSONs could not be resolved to full UUID)")
    
    return hevl_blocks

uuid8_to_full = build_uuid8_to_full(df_nb06_index)
hevl_json_blocks = load_hevl_jsons(EXTRACTED_DIR, uuid8_to_full)

UUID8→full mapping: 13,152 entries
Loaded HEVL JSON blocks for 12594 datasets:
  hazard: 2791 blocks
  exposure: 11522 blocks
  vulnerability: 3442 blocks
  loss: 705 blocks
  (11570 HEVL JSONs could not be resolved to full UUID)


In [8]:
"""
3.2 Verify NB 06 Record Lookup

NB 06 records are already indexed by full HDX UUID (loaded in cell 4).
This cell verifies the lookup is working and shows coverage stats.
"""

print(f"NB 06 records indexed by full UUID: {len(nb06_records):,}")

# Check overlap with HEVL-flagged datasets
hevl_uuids = set(df_hevl['id'].values)
nb06_uuids = set(nb06_records.keys())

overlap = hevl_uuids & nb06_uuids
hevl_only = hevl_uuids - nb06_uuids
nb06_only = nb06_uuids - hevl_uuids

print(f"\nCoverage check:")
print(f"  Datasets with HEVL signals: {len(hevl_uuids):>8,}")
print(f"  NB 06 records available:    {len(nb06_uuids):>8,}")
print(f"  Overlap (will integrate):   {len(overlap):>8,}")
print(f"  HEVL but no NB 06 record:   {len(hevl_only):>8,}")
print(f"  NB 06 but no HEVL signal:   {len(nb06_only):>8,}")

if hevl_only:
    print(f"\n  Sample HEVL-only UUIDs (first 5): {list(hevl_only)[:5]}")

NB 06 records indexed by full UUID: 13,152

Coverage check:
  Datasets with HEVL signals:   22,473
  NB 06 records available:      13,152
  Overlap (will integrate):     12,843
  HEVL but no NB 06 record:      9,630
  NB 06 but no HEVL signal:        309

  Sample HEVL-only UUIDs (first 5): ['1cfe7b03-4a3a-4947-bf72-c0337a36a9ef', '6628359d-c7c9-40aa-a07f-22e22755d59d', 'e362a580-5075-4525-84f0-ec9500f6e09d', 'da611749-4214-4945-bd97-7f97ffa9ac62', 'f7ad9387-0fd0-4cbb-a871-48ff25bf1de2']


## 4. Integration Logic

In [9]:
"""
4.1 Determine risk_data_type and Filename Prefix

Naming convention priority (highest → lowest):
  loss > vulnerability > exposure > hazard

Examples:
  H only              → rdls_hzd
  E only              → rdls_exp
  H + E               → rdls_exp   (exposure > hazard)
  H + L               → rdls_lss   (loss > hazard)
  E + L               → rdls_lss   (loss > exposure)
  E + V               → rdls_vln   (vulnerability > exposure)
  H + V               → rdls_vln   (vulnerability > hazard)
  V + L               → rdls_lss   (loss > vulnerability)
  H + E + V           → rdls_vln   (vulnerability > exposure > hazard)
  H + E + L           → rdls_lss   (loss > exposure > hazard)
  E + V + L           → rdls_lss   (loss > vulnerability > exposure)
  H + V + L           → rdls_lss   (loss > vulnerability > hazard)
  H + E + V + L       → rdls_lss   (loss > vulnerability > exposure > hazard)

Standalone constraint (configurable):
  V only (no H or E)  → INVALID — vulnerability needs hazard or exposure context
  L only (no H or E)  → INVALID — loss needs hazard or exposure context
  V + L (no H or E)   → INVALID — same reason
  Set REQUIRE_HE_FOR_VL = False to disable this constraint.
"""

# Order for building risk_data_type list (canonical HEVL order)
COMPONENT_ORDER = ['hazard', 'exposure', 'vulnerability', 'loss']

# Priority order for filename prefix: loss > vulnerability > exposure > hazard
PREFIX_PRIORITY = ['loss', 'vulnerability', 'exposure', 'hazard']

PREFIX_MAP = {
    'hazard': 'rdls_hzd',
    'exposure': 'rdls_exp',
    'vulnerability': 'rdls_vln',
    'loss': 'rdls_lss',
}

# ── Standalone constraint (modular toggle) ────────────────────────────
# When True: V and L cannot stand alone — they must be paired with H or E.
# Combinations like V-only, L-only, or V+L (without H or E) are invalid.
# Set to False if the team decides standalone V or L is acceptable.
REQUIRE_HE_FOR_VL = True


def determine_risk_data_types(flags: Dict[str, bool]) -> List[str]:
    """Determine risk_data_type list from boolean flags (canonical HEVL order)."""
    types = []
    for comp in COMPONENT_ORDER:
        if flags.get(f'has_{comp}', False):
            types.append(comp)
    return types


def validate_component_combination(risk_types: List[str]) -> Tuple[bool, str]:
    """
    Validate whether a component combination is allowed.

    Returns (is_valid, reason).
    When REQUIRE_HE_FOR_VL is True, vulnerability and loss cannot appear
    without at least one of hazard or exposure.
    """
    if not REQUIRE_HE_FOR_VL:
        return True, 'ok'

    has_h_or_e = ('hazard' in risk_types) or ('exposure' in risk_types)
    has_v_or_l = ('vulnerability' in risk_types) or ('loss' in risk_types)

    if has_v_or_l and not has_h_or_e:
        present = [c for c in risk_types]
        return False, f"standalone_vl_invalid({'+'.join(present)})"

    return True, 'ok'


def determine_filename_prefix(risk_types: List[str]) -> str:
    """
    Determine filename prefix based on naming convention priority.

    Priority: loss > vulnerability > exposure > hazard
    The prefix reflects the HIGHEST-priority component present.
    """
    for comp in PREFIX_PRIORITY:
        if comp in risk_types:
            return PREFIX_MAP[comp]
    return 'rdls_unk'


# ── Verify all 15 possible HEVL combinations ─────────────────────────
_test_cases = [
    # Single component
    (['hazard'],                                      'rdls_hzd', True),
    (['exposure'],                                    'rdls_exp', True),
    (['vulnerability'],                               'rdls_vln', False),  # standalone V invalid
    (['loss'],                                        'rdls_lss', False),  # standalone L invalid
    # Two components
    (['hazard', 'exposure'],                          'rdls_exp', True),
    (['hazard', 'vulnerability'],                     'rdls_vln', True),
    (['hazard', 'loss'],                              'rdls_lss', True),
    (['exposure', 'vulnerability'],                   'rdls_vln', True),
    (['exposure', 'loss'],                            'rdls_lss', True),
    (['vulnerability', 'loss'],                       'rdls_lss', False),  # standalone V+L invalid
    # Three components
    (['hazard', 'exposure', 'vulnerability'],         'rdls_vln', True),
    (['hazard', 'exposure', 'loss'],                  'rdls_lss', True),
    (['hazard', 'vulnerability', 'loss'],             'rdls_lss', True),
    (['exposure', 'vulnerability', 'loss'],           'rdls_lss', True),
    # All four
    (['hazard', 'exposure', 'vulnerability', 'loss'], 'rdls_lss', True),
]

_all_pass = True
for _types, _expected_prefix, _expected_valid in _test_cases:
    _actual_prefix = determine_filename_prefix(_types)
    _actual_valid, _ = validate_component_combination(_types)
    if _actual_prefix != _expected_prefix:
        print(f"  PREFIX FAIL: {_types} → {_actual_prefix} (expected {_expected_prefix})")
        _all_pass = False
    if _actual_valid != _expected_valid:
        print(f"  VALID  FAIL: {_types} → valid={_actual_valid} (expected {_expected_valid})")
        _all_pass = False

if _all_pass:
    print(f"Integration helpers defined. All {len(_test_cases)} combinations verified ✓")
    print(f"  Prefix priority:  loss > vulnerability > exposure > hazard")
    print(f"  Standalone guard:  REQUIRE_HE_FOR_VL = {REQUIRE_HE_FOR_VL}")
else:
    print("WARNING: Some test cases failed!")

Integration helpers defined. All 15 combinations verified ✓
  Prefix priority:  loss > vulnerability > exposure > hazard
  Standalone guard:  REQUIRE_HE_FOR_VL = True


In [10]:
"""
4.2 Integrate: Merge HEVL Blocks into NB 06 Base Records

For each dataset with HEVL signals:
1. Look up the NB 06 base record by full HDX UUID
2. Deep-copy the base record
3. Determine risk_data_type and validate the combination
4. Update the RDLS id and filename prefix (loss > vulnerability > exposure > hazard)
5. Insert HEVL blocks from NB 09-11 JSONs (if available)
6. RECONCILE risk_data_type with actual blocks present
7. Re-derive filename prefix from reconciled types
8. Append HDX provenance note to description
"""

HDX_PROVENANCE_NOTE = (
    "[Source: This metadata record was automatically extracted from the "
    "Humanitarian Data Exchange (HDX) at https://data.humdata.org]"
)

def integrate_record(
    hdx_uuid: str,
    flags: Dict[str, bool],
    nb06_entry: Optional[Dict],
    hevl_blocks: Optional[Dict[str, Any]],
) -> Tuple[Optional[Dict], str]:
    """
    Produce an integrated RDLS record.

    Args:
        hdx_uuid: Full 36-char HDX dataset UUID
        flags: Boolean HEVL detection flags
        nb06_entry: NB 06 record entry (dict with 'data', 'rdls_id', etc.)
        hevl_blocks: HEVL JSON blocks (dict with 'hazard', 'exposure', etc.)

    Returns (rdls_record, status_message).
    """
    risk_types = determine_risk_data_types(flags)

    if not risk_types:
        return None, 'no_hevl_signals'

    # --- Validate component combination ---
    is_valid, reason = validate_component_combination(risk_types)
    if not is_valid:
        return None, reason

    # --- Base record from NB 06 ---
    if nb06_entry is not None:
        record = deepcopy(nb06_entry['data'])
    else:
        # No NB 06 record: skip (NB 06 is authoritative for general metadata)
        return None, 'no_nb06_record'

    ds = record['datasets'][0]

    # --- Update risk_data_type ---
    ds['risk_data_type'] = risk_types

    # --- Update RDLS id prefix to reflect primary component ---
    # Priority: loss > vulnerability > exposure > hazard
    old_id = ds.get('id', '')
    new_prefix = determine_filename_prefix(risk_types)

    # Replace the existing prefix pattern: rdls_XXX-hdx_ -> new_prefix-hdx_
    new_id = re.sub(r'^rdls_\w+-hdx_', f'{new_prefix}-hdx_', old_id)
    if new_id == old_id and not old_id.startswith(new_prefix):
        # Fallback: just prepend new prefix with UUID
        uuid8 = hdx_uuid[:8]
        new_id = f"{new_prefix}-hdx_{uuid8}"
    ds['id'] = new_id

    # --- Move "links" to end (insert HEVL blocks before it) ---
    links_value = ds.pop('links', None)

    # --- Insert HEVL blocks from JSON files ---
    if hevl_blocks:
        for comp in COMPONENT_ORDER:
            if comp in hevl_blocks:
                ds[comp] = deepcopy(hevl_blocks[comp])

    # --- RECONCILE risk_data_type with actual blocks present ---
    # CSV flags may declare components that don't have actual JSON blocks.
    # Only list components that are actually present in the record.
    actual_types = [c for c in COMPONENT_ORDER if c in ds]
    if not actual_types:
        return None, 'no_actual_hevl_blocks'

    # Re-validate the reconciled combination
    is_valid_actual, reason_actual = validate_component_combination(actual_types)
    if not is_valid_actual:
        return None, f'reconciled_{reason_actual}'

    # Update risk_data_type to match reality
    ds['risk_data_type'] = actual_types

    # Re-derive filename prefix from reconciled types
    reconciled_prefix = determine_filename_prefix(actual_types)
    ds['id'] = re.sub(r'^rdls_\w+-hdx_', f'{reconciled_prefix}-hdx_', ds.get('id', ''))

    # --- Re-add "links" at the very end of the dataset object ---
    if links_value is not None:
        ds['links'] = links_value

    # --- Append HDX provenance note to description ---
    hdx_url = f"https://data.humdata.org/dataset/{hdx_uuid}"
    provenance = (
        f"{HDX_PROVENANCE_NOTE} "
        f"[Original dataset: {hdx_url}]"
    )
    existing_desc = (ds.get('description') or '').rstrip()
    if existing_desc:
        if existing_desc[-1] not in '.!?:;)"':  # Add period if no terminal punctuation
            existing_desc += '.'
        ds['description'] = f"{existing_desc} {provenance}"
    else:
        ds['description'] = provenance

    return record, 'ok'

print("Integration function defined.")

Integration function defined.


In [11]:
"""
4.3 Process All Records
"""

def process_integration(
    df_hevl: pd.DataFrame,
    nb06_records: Dict[str, Dict],
    hevl_json_blocks: Dict[str, Dict],
    output_dir: Path,
) -> Dict[str, Any]:
    """
    Process all datasets with HEVL signals and write integrated records.

    All lookups use full 36-char HDX UUIDs as the join key.
    """
    stats = {
        'total': len(df_hevl),
        'integrated': 0,
        'with_json_blocks': 0,
        'flags_only': 0,
        'no_nb06': 0,
        'no_actual_blocks': 0,
        'reconciled_vl_invalid': 0,
        'standalone_vl_invalid': 0,
        'errors': 0,
        'by_type': {c: 0 for c in COMPONENT_ORDER},
        'by_type_declared': {c: 0 for c in COMPONENT_ORDER},
        'by_prefix': {},
    }

    skipped_records = []

    iterator = tqdm(df_hevl.iterrows(), total=len(df_hevl), desc="Integrating") if HAS_TQDM else df_hevl.iterrows()

    for idx, row in iterator:
        hdx_uuid = row['id']  # Full 36-char UUID from extraction CSV

        flags = {
            'has_hazard': bool(row.get('has_hazard', False)),
            'has_exposure': bool(row.get('has_exposure', False)),
            'has_vulnerability': bool(row.get('has_vulnerability', False)),
            'has_loss': bool(row.get('has_loss', False)),
        }

        # Lookup by FULL UUID
        nb06_entry = nb06_records.get(hdx_uuid)
        blocks = hevl_json_blocks.get(hdx_uuid)

        try:
            record, status = integrate_record(hdx_uuid, flags, nb06_entry, blocks)

            if record is None:
                if status == 'no_nb06_record':
                    stats['no_nb06'] += 1
                elif status == 'no_actual_hevl_blocks':
                    stats['no_actual_blocks'] += 1
                elif 'reconciled_' in status:
                    stats['reconciled_vl_invalid'] += 1
                elif 'standalone_vl_invalid' in status:
                    stats['standalone_vl_invalid'] += 1
                skipped_records.append({'id': hdx_uuid, 'reason': status})
                continue

            # Determine output filename from the integrated record's RDLS id
            rdls_id = record['datasets'][0].get('id', '')
            filename = f"{rdls_id}.json" if rdls_id else f"rdls_unk-hdx_{hdx_uuid[:8]}.json"

            # Track prefix distribution
            prefix = rdls_id.split('-')[0] if '-' in rdls_id else 'rdls_unk'
            stats['by_prefix'][prefix] = stats['by_prefix'].get(prefix, 0) + 1

            # Write
            with open(output_dir / filename, 'w', encoding='utf-8') as f:
                json.dump(record, f, indent=2, ensure_ascii=False)

            stats['integrated'] += 1
            if blocks:
                stats['with_json_blocks'] += 1
            else:
                stats['flags_only'] += 1

            # Track declared types (from CSV flags) for comparison
            for comp in COMPONENT_ORDER:
                if flags.get(f'has_{comp}', False):
                    stats['by_type_declared'][comp] += 1

            # Track actual types (after reconciliation)
            risk_types = record['datasets'][0].get('risk_data_type', [])
            for rt in risk_types:
                stats['by_type'][rt] += 1

        except Exception as e:
            stats['errors'] += 1
            skipped_records.append({'id': hdx_uuid, 'reason': f'error: {e}'})

    # Save skipped records
    if skipped_records:
        pd.DataFrame(skipped_records).to_csv(
            output_dir / 'integration_skipped.csv', index=False
        )

    return stats

print("Processing function defined.")

Processing function defined.


## 5. Run Integration

In [12]:
"""
5.1 Clean Previous Outputs and Run Integration

Removes stale output files before writing fresh records.
Controlled by CLEANUP_MODE variable (set in cell 1.2).
"""

def clean_previous_outputs(output_dir, patterns, label, mode="replace"):
    """
    Clean stale output files before writing new ones.

    Args:
        output_dir: Path to the output directory
        patterns: list of glob patterns to match
        label: human-readable label for display
        mode: "replace" | "prompt" | "skip" | "abort"

    Returns:
        dict with keys: deleted (int), skipped (bool)
    """
    result = {"deleted": 0, "skipped": False}

    # Find all matching files
    targets = {}
    for pattern in patterns:
        matches = sorted(output_dir.glob(pattern))
        if matches:
            targets[pattern] = matches

    total = sum(len(files) for files in targets.values())

    # Nothing to clean
    if total == 0:
        print(f"Output cleanup [{label}]: Directory is clean.")
        return result

    # Build summary lines
    summary = []
    for pattern, files in targets.items():
        summary.append(f"  {pattern:30s}: {len(files):,} files")

    # --- Mode: skip ---
    if mode == "skip":
        print(f"Output cleanup [{label}]: SKIPPED ({total:,} existing files kept)")
        result["skipped"] = True
        return result

    # --- Mode: abort ---
    if mode == "abort":
        print(f"Output cleanup [{label}]: {total:,} stale files found!")
        for line in summary:
            print(line)
        raise RuntimeError(
            f"Abort: {total:,} stale files in {output_dir.name}/. "
            f"Clear manually or set CLEANUP_MODE='replace'."
        )

    # --- Mode: prompt ---
    if mode == "prompt":
        print(f"\n{'=' * 60}")
        print(f"OUTPUT CLEANUP — {label}")
        print(f"{'=' * 60}")
        print(f"Found {total:,} existing output files in {output_dir.name}/:\n")
        for line in summary:
            print(line)
        print()
        print("Options:")
        print("  [R]eplace  - Delete all and write fresh")
        print("  [S]kip     - Keep existing, write on top")
        print("  [A]bort    - Stop notebook execution")
        print()

        choice = input("Choose [R/S/A]: ").strip().lower()

        if choice in ("a", "abort"):
            raise RuntimeError("Cleanup aborted by user.")
        elif choice in ("s", "skip"):
            print("Cleanup: SKIPPED by user.")
            result["skipped"] = True
            return result
        elif choice not in ("r", "replace", ""):
            print(f"Unrecognized '{choice}'. Defaulting to Replace.")

    # --- Mode: replace (default) or user chose Replace ---
    print(f"Output cleanup [{label}]:")
    for line in summary:
        print(line)

    for pattern, files in targets.items():
        for f in files:
            try:
                f.unlink()
                result["deleted"] += 1
            except Exception as e:
                print(f"  WARNING: Could not delete {f.name}: {e}")

    print(f"Cleaned {result['deleted']:,} files. Ready for fresh output.\n")
    return result


# ── Clean stale output ────────────────────────────────────────────────
clean_previous_outputs(
    OUTPUT_DIR,
    patterns=[
        "rdls_*.json",
        "rdls_index.csv",
        "rdls_index.jsonl",
        "integration_skipped.csv",
    ],
    label="NB 12 Integrated Records",
    mode=CLEANUP_MODE,
)

# ── Run integration ──────────────────────────────────────────────────
print(f"Integrating {len(df_hevl):,} datasets with HEVL signals...")
print(f"  NB 06 base records available: {len(nb06_records):,}")
print(f"  HEVL JSON blocks available:   {len(hevl_json_blocks):,}")
print(f"  Standalone V/L guard:         REQUIRE_HE_FOR_VL = {REQUIRE_HE_FOR_VL}")
print()

stats = process_integration(df_hevl, nb06_records, hevl_json_blocks, OUTPUT_DIR)

print(f"\n{'='*60}")
print("INTEGRATION RESULTS")
print(f"{'='*60}")
print(f"Total candidates:           {stats['total']:>8,}")
print(f"Successfully integrated:    {stats['integrated']:>8,}")
print(f"  With JSON HEVL blocks:    {stats['with_json_blocks']:>8,}")
print(f"  Flags only (no JSON):     {stats['flags_only']:>8,}")
print(f"Skipped (no NB 06):         {stats['no_nb06']:>8,}")
print(f"Skipped (no actual blocks): {stats.get('no_actual_blocks', 0):>8,}")
print(f"Skipped (reconciled V/L):   {stats.get('reconciled_vl_invalid', 0):>8,}")
print(f"Skipped (standalone V/L):   {stats['standalone_vl_invalid']:>8,}")
print(f"Errors:                     {stats['errors']:>8,}")

print(f"\nBy component type (in integrated records):")
for comp, count in stats['by_type'].items():
    print(f"  {comp:20s}: {count:>6,}")

print(f"\nBy filename prefix:")
for prefix in sorted(stats['by_prefix'], key=lambda k: -stats['by_prefix'][k]):
    count = stats['by_prefix'][prefix]
    print(f"  {prefix:12s}: {count:>6,}")

# --- Reconciliation summary ---
if "by_type_declared" in stats:
    print()
    print("=" * 60)
    print("RECONCILIATION (declared vs actual risk_data_type)")
    print("=" * 60)
    print(f"  {'Component':<20s} {'Declared':>10s} {'Actual':>10s} {'Dropped':>10s}")
    for comp in ["hazard", "exposure", "vulnerability", "loss"]:
        declared = stats["by_type_declared"].get(comp, 0)
        actual = stats["by_type"].get(comp, 0)
        dropped = declared - actual
        print(f"  {comp:<20s} {declared:>10,} {actual:>10,} {dropped:>10,}")


Output cleanup [NB 12 Integrated Records]:
  rdls_*.json                   : 12,577 files
  rdls_index.csv                : 1 files
  rdls_index.jsonl              : 1 files
  integration_skipped.csv       : 1 files
Cleaned 12,580 files. Ready for fresh output.

Integrating 22,473 datasets with HEVL signals...
  NB 06 base records available: 13,152
  HEVL JSON blocks available:   12,594
  Standalone V/L guard:         REQUIRE_HE_FOR_VL = True



Integrating:   0%|          | 0/22473 [00:00<?, ?it/s]


INTEGRATION RESULTS
Total candidates:             22,473
Successfully integrated:      12,583
  With JSON HEVL blocks:      12,583
  Flags only (no JSON):            0
Skipped (no NB 06):            9,536
Skipped (no actual blocks):        5
Skipped (reconciled V/L):          0
Skipped (standalone V/L):        349
Errors:                            0

By component type (in integrated records):
  hazard              :  2,791
  exposure            : 11,522
  vulnerability       :  3,431
  loss                :    705

By filename prefix:
  rdls_exp    :  7,563
  rdls_vln    :  3,400
  rdls_hzd    :    915
  rdls_lss    :    705

RECONCILIATION (declared vs actual risk_data_type)
  Component              Declared     Actual    Dropped
  hazard                    2,791      2,791          0
  exposure                 12,155     11,522        633
  vulnerability             3,431      3,431          0
  loss                      3,726        705      3,021


## 6. Generate Index and Summary

In [13]:
"""
6.1 Generate RDLS Index of Integrated Records
"""

def generate_rdls_index(output_dir: Path) -> pd.DataFrame:
    """Scan integrated JSON files and build an index DataFrame."""
    records = []
    
    for filepath in sorted(output_dir.glob('rdls_*.json')):
        try:
            with open(filepath, 'r', encoding='utf-8') as f:
                data = json.load(f)
            
            ds = data.get('datasets', [{}])[0]
            
            records.append({
                'filename': filepath.name,
                'id': ds.get('id', ''),
                'title': (ds.get('title', '') or '')[:120],
                'risk_data_type': '|'.join(ds.get('risk_data_type', [])),
                'has_hazard': 'hazard' in ds,
                'has_exposure': 'exposure' in ds,
                'has_vulnerability': 'vulnerability' in ds,
                'has_loss': 'loss' in ds,
                'license': ds.get('license', ''),
                'resource_count': len(ds.get('resources', [])),
                'country_count': len(ds.get('spatial', {}).get('countries', [])),
            })
        except Exception:
            pass
    
    df = pd.DataFrame(records)
    df.to_csv(output_dir / 'rdls_index.csv', index=False)
    
    # JSONL format
    with open(output_dir / 'rdls_index.jsonl', 'w', encoding='utf-8') as f:
        for rec in records:
            f.write(json.dumps(rec, ensure_ascii=False) + '\n')
    
    return df

df_index = generate_rdls_index(OUTPUT_DIR)
print(f"Index generated: {len(df_index)} records")
print(f"Saved to: {OUTPUT_DIR / 'rdls_index.csv'}")

Index generated: 12577 records
Saved to: /mnt/c/Users/benny/OneDrive/Documents/Github/hdx-metadata-crawler/hdx_dataset_metadata_dump/rdls/integrated/rdls_index.csv


In [14]:
"""
6.2 Summary Report
"""

if len(df_index) > 0:
    print("=" * 60)
    print("INTEGRATED RDLS RECORDS SUMMARY")
    print("=" * 60)
    print(f"\nTotal records: {len(df_index)}")
    
    print(f"\nBy risk_data_type combination:")
    for combo, count in df_index['risk_data_type'].value_counts().items():
        print(f"  {combo:40s}: {count:>6,}")
    
    print(f"\nComponent coverage:")
    print(f"  With hazard block:        {df_index['has_hazard'].sum():>6,}")
    print(f"  With exposure block:      {df_index['has_exposure'].sum():>6,}")
    print(f"  With vulnerability block: {df_index['has_vulnerability'].sum():>6,}")
    print(f"  With loss block:          {df_index['has_loss'].sum():>6,}")
    
    # Records with actual HEVL JSON blocks vs flags-only
    has_any_block = df_index['has_hazard'] | df_index['has_exposure'] | df_index['has_vulnerability'] | df_index['has_loss']
    print(f"\n  Records with HEVL JSON blocks: {has_any_block.sum():>6,}")
    print(f"  Records flags-only (no block): {(~has_any_block).sum():>6,}")
else:
    print("No integrated records found.")

INTEGRATED RDLS RECORDS SUMMARY

Total records: 12577

By risk_data_type combination:
  exposure                                :  6,449
  exposure|vulnerability                  :  3,340
  hazard|exposure                         :  1,113
  hazard                                  :    914
  hazard|exposure|loss                    :    527
  hazard|loss                             :    145
  hazard|exposure|vulnerability           :     57
  hazard|exposure|vulnerability|loss      :     31
  hazard|vulnerability                    :      1

Component coverage:
  With hazard block:         2,788
  With exposure block:      11,517
  With vulnerability block:  3,429
  With loss block:             703

  Records with HEVL JSON blocks: 12,577
  Records flags-only (no block):      0


In [15]:
print(f"\nNotebook completed: {datetime.now().isoformat()}")
print(f"Output directory: {OUTPUT_DIR}")
print(f"Records generated: {len(df_index)}")


Notebook completed: 2026-02-11T18:25:31.370201
Output directory: /mnt/c/Users/benny/OneDrive/Documents/Github/hdx-metadata-crawler/hdx_dataset_metadata_dump/rdls/integrated
Records generated: 12577


## End of Code